In [2]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split

In [2]:
# def read_excel_file(file_path: str, header: int = None) -> pd.DataFrame:
#     csv_file = file_path.replace(".xlsx", ".csv")

#     if not os.path.exists(csv_file):
#         print("Converting excel to csv...")
#         if header:
#             df = pd.read_excel(file_path, header=header)
#         else:
#             df = pd.read_excel(file_path)

#         df.to_csv(csv_file, index=False)
#         print(f"  {file_path} -> {csv_file}")
#         return df
#     else:
#         print(f"  Reading {csv_file}")
#         return pd.read_csv(csv_file, low_memory=False)

In [6]:
# ROOT_DIR = "dsets"
# RANDOM_STATE = 110

# X_Dam = read_excel_file(os.path.join(ROOT_DIR, "Dam dispensing.xlsx"), header=1)

# X_AutoClave = read_excel_file(
#     os.path.join(ROOT_DIR, "Auto clave.xlsx"), header=1
# )

# X_Fill1 = read_excel_file(
#     os.path.join(ROOT_DIR, "Fill1 dispensing.xlsx"), header=1
# )

# X_Fill2 = read_excel_file(
#     os.path.join(ROOT_DIR, "Fill2 dispensing.xlsx"), header=1
# )

# y = pd.read_csv(os.path.join(ROOT_DIR, "train_y.csv"))

Converting excel to csv...
  dsets/Dam dispensing.xlsx -> dsets/Dam dispensing.csv
Converting excel to csv...
  dsets/Auto clave.xlsx -> dsets/Auto clave.csv
Converting excel to csv...
  dsets/Fill1 dispensing.xlsx -> dsets/Fill1 dispensing.csv
Converting excel to csv...
  dsets/Fill2 dispensing.xlsx -> dsets/Fill2 dispensing.csv


In [1]:
import pandas as pd

ROOT_DIR = "data"
df = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
df.dropna(axis=1, how='all', inplace=True)

"""
Data process part 
"""

output_path = 'data/train_null.csv'
df.to_csv(output_path, index=False)
print(f"Processed data saved to {output_path}")

NameError: name 'os' is not defined

In [ ]:
#PART 2. Remove the features with high correlation
#It looks for columns with identical distribution of unique values and checks for identical mappings between these values 
def reduce_dataframe(df):
#2-1. Get the ratio of the unique values in the given series as a sorted array
    def get_value_counts_ratio(series):
        value_counts = series.value_counts(normalize=True) 
        return value_counts.sort_values().values 

#2-2. Check if there's identical mapping between the unique values of two columns 
# Returns 'true' if one unique value in 'col1' maps to one unique value in 'col2'
    def check_value_mapping(df, col1, col2):
        unique_column_1 = df[col1].unique()
        unique_column_2 = df[col2].unique()

        if len(unique_column_1) != len(unique_column_2) : return False

        value_mapping = {}
        #Check if there's consistent mapping between values in col1 and col2 
        #maybe needs some margin??
        for val1 in unique_column_1:
            consistent_values = df[df[col1] == val1][col2].unique()
            if len(consistent_values) != 1 : 
                return False
            value_mapping [val1] = consistent_values[0]
        
        #Check if there's matching proportion of occurences of values in col1 and col2
        for val1 in unique_column_1: 
            ratio_1 = (df[col1] == val1).mean()
            ratio_2 = (df[col2] == value_mapping[val1]).mean()
            if ratio_1 != ratio_2:
                return False

        return True

#2-3. Idenify columns to remove 
    def compare_all_features(df):
        #Calculate Ratios 
        ratios = {column: get_value_counts_ratio(df[column]) for column in df.columns}
        similar_columns_dict = {column: [] for column in df.columns}

        #Update 'similar_columns_dict' with columns having identical ratios
        columns = list(ratios.keys())
        for i in range(len(columns)):
            for j in range(i + 1, len(columns)):
                if np.array_equal(ratios[columns[i]], ratios[columns[j]]):
                    similar_columns_dict[columns[i]].append(columns[j])

        #Update 'comparisions' fro identical value mappings among columns with similar ratios
        comparisons = []
        for key, values in similar_columns_dict.items():
            for value in values:
                if check_value_mapping(df, key, value):
                    comparisons.append((key, value))
                    

        if comparisons:
            print("다음 피쳐 쌍은 고유값 비율과 양상이 동일합니다:")
            for base, compare in comparisons:
                print(f"'{base}'와(과) '{compare}'")
        else:
            print("모든 피쳐의 고유값 비율과 양상이 동일하지 않습니다.")

        return comparisons

    comparisons = compare_all_features(df)
    columns_to_remove = set()
    for _, col_to_remove in comparisons:
        columns_to_remove.add(col_to_remove)

    df = df.drop(columns=columns_to_remove)
    global df_columns
    df_columns = df.columns
    return df

In [ ]:
def preprocess_train_dataframe(df):
    df = preprocess_unique_value(df)
    df = reduce_dataframe(df)
    return df

In [ ]:
# Added part
df_p = pd.read_csv('data/train_null.csv')
df_pre = preprocess_train_dataframe(df_p)
df_pre.head(2)
pre_path = 'data/train_preprocess.csv'
df_pre.to_csv(pre_path, index=False)

In [ ]:
pre_path = 'data/train_preprocess.csv'
df_pre.to_csv(pre_path, index=False)

In [ ]:
def is_column_to_drop(col):
    unique_values = col.dropna().unique()
    return len(unique_values) == 0 or (len(unique_values) == 1 and unique_values[0] == 'OK') or set(unique_values) == {'OK'}

columns_to_drop = [col for col in df_pre.columns if is_column_to_drop(df_pre[col])]
df_pre.drop(columns=columns_to_drop, inplace=True)


output_path = 'data/train_preprocess2.csv'
df_pre.to_csv(output_path, index=False)

##### Data check

In [22]:
X_Dam[["Set ID", "Collect Date"]].head(20)

,Set ID,Collect Date
0,OP753345013050000002,2023-05-04 08:57:23
1,OP753345013050000003,2023-05-04 09:11:35
2,OP753345013050000004,2023-05-04 09:13:19
3,OP753345013050000005,2023-05-04 09:15:24
4,OP753345013050000006,2023-05-04 09:17:27
5,OP753345013050000007,2023-05-04 09:19:31
6,OP753345013050000008,2023-05-04 09:21:34
7,OP753345013050000009,2023-05-04 09:25:02
8,OP753345013050000011,2023-05-04 11:19:55
9,OP753345013050000015,2023-05-04 11:29:37


In [25]:
X_Fill1[["Set ID", "Collect Date"]].head(20)

,Set ID,Collect Date
0,OP753345013050000002,2023-05-04 08:59:27
1,OP753345013050000003,2023-05-04 09:13:39
2,OP753345013050000004,2023-05-04 09:15:42
3,OP753345013050000005,2023-05-04 09:17:46
4,OP753345013050000006,2023-05-04 09:19:50
5,OP753345013050000007,2023-05-04 09:21:54
6,OP753345013050000008,2023-05-04 09:25:21
7,OP753345013050000009,2023-05-04 09:27:25
8,OP753345013050000011,2023-05-04 11:22:00
9,OP753345013050000015,2023-05-04 11:31:41


In [24]:
X_Fill2[["Set ID", "Collect Date"]].head(20)

,Set ID,Collect Date
0,OP753345013050000002,2023-05-04 09:00:48
1,OP753345013050000003,2023-05-04 09:15:00
2,OP753345013050000004,2023-05-04 09:17:04
3,OP753345013050000005,2023-05-04 09:19:08
4,OP753345013050000006,2023-05-04 09:21:11
5,OP753345013050000007,2023-05-04 09:24:39
6,OP753345013050000008,2023-05-04 09:26:42
7,OP753345013050000009,2023-05-04 09:28:46
8,OP753345013050000011,2023-05-04 11:23:21
9,OP753345013050000015,2023-05-04 11:33:02


In [26]:
X_AutoClave[["Set ID", "Collect Date"]].head(20)

,Set ID,Collect Date
0,OP753345013050000002,2023-05-04 09:29:00
1,OP753345013050000003,2023-05-04 09:49:12
2,OP753345013050000004,2023-05-04 09:49:12
3,OP753345013050000005,2023-05-04 09:49:12
4,OP753345013050000006,2023-05-04 09:49:12
5,OP753345013050000007,2023-05-04 10:03:02
6,OP753345013050000008,2023-05-04 10:03:02
7,OP753345013050000009,2023-05-04 10:03:02
8,OP753345013050000010,2023-05-05 13:36:50
9,OP753345013050000011,2023-05-04 11:50:09


In [37]:
X_AutoClave["Judge Value.3"].unique() # 3만 NG 존재

array(['OK', 'NG'], dtype=object)

In [38]:
X_Fill1[""]

Index(['Wip Line', 'Process Desc.', 'Equipment', 'Model.Suffix', 'Workorder',
       'LOT ID', 'Set ID', 'Box ID', 'Collect Date', 'Insp. Seq No.',
       ...
       'Collect Result.27', 'Unit Time.27', 'Judge Value.27',
       'Collect Result.28', 'Unit Time.28', 'Judge Value.28',
       'Collect Result.29', 'Unit Time.29', 'Judge Value.29', 'Unnamed: 101'],
      dtype='object', length=102)

In [42]:
X_Fill1 = read_excel_file(
    os.path.join(ROOT_DIR, "Fill1 dispensing.xlsx"), header=1
)

Converting excel to csv...
  dsets/Fill1 dispensing.xlsx -> dsets/Fill1 dispensing.csv


In [45]:
X_Fill1.columns

Index(['Wip Line', 'Process Desc.', 'Equipment', 'Model.Suffix', 'Workorder',
       'LOT ID', 'Set ID', 'Box ID', 'Collect Date', 'Insp. Seq No.',
       ...
       'Production Qty', 'Unit Time.27', 'Judge Value.27', 'Receip No',
       'Unit Time.28', 'Judge Value.28', 'WorkMode', 'Unit Time.29',
       'Judge Value.29', 'Unnamed: 101'],
      dtype='object', length=102)

In [48]:
X_Fill2.columns

Index(['Wip Line', 'Process Desc.', 'Equipment', 'Model.Suffix', 'Workorder',
       'LOT ID', 'Set ID', 'Box ID', 'Collect Date', 'Insp. Seq No.',
       ...
       'Collect Result.37', 'Unit Time.37', 'Judge Value.37',
       'Collect Result.38', 'Unit Time.38', 'Judge Value.38',
       'Collect Result.39', 'Unit Time.39', 'Judge Value.39', 'Unnamed: 131'],
      dtype='object', length=132)